In [4]:
import pandas as pd
corpus_path = './df_terms_190327.pkl'
df = pd.read_pickle(corpus_path)
df.head()

,sr_no,customer_text,gs_text,customer_terms,gs_terms
0,S121383683,특상을 주문했는데 중상이 오배송된 건과 관련하여 차액을 지급하는 걸로 알고있었는데 ...,안녕하세요 고객님 GS SHOP 선정민 입니다.기대하시고 주문하셨을텐데 … 만족을 ...,특상 주문 중상 오배송 건과일 차액 지급 알 있 적립 금액 이해 안 돼서 되 같 여...,안녕 gs shop 선 정민 주문 못해 특상 19 975 원 중상 17 765 원 ...
1,S121389801,한개 상했어요,안녕하세요 고객님 GS SHOP 이윤복 입니다.불편드려 죄송합니다.내일 공휴일로 인...,한 개 상했 상하,안녕 gs shop 이윤 옷 불편 드려 공휴일 인해 인하 불가 월요일 오후 6 시 ...
2,S092377332,주문취소하고싶어요,네 고객님!주문하신 상품은 취소해드리겠습니다.다만 상품이 이미 상품이 발송되어 배송...,주문취소 주문 취소 싶,네 주문 취소 다만 배송기사 배송 기사 시 택배기사 기사 취소 후 꼭 거부 쥬시 바...
3,S092377428,주문작업중 직택배 직반 주문번호 가주문 단가조정 원,None,주문 작업 직택배 직 반 주문번호 주문 가주문 단가 다 조정 원,None
4,S092385617,협력사 위드넷 접수 협력사 접수자명 주식회사 다드림 농업회사법인 위드넷 접수시간 ...,None,협력사 위드넷 협력사 자명 주식회사 주식 회사 다 농업 회사 법인 위드넷 제목 미출...,None


In [20]:
documents = df.customer_terms.tolist()
documents = [doc.split() if doc else [""] for doc in documents]
documents[:2]

[['특상',
  '주문',
  '중상',
  '오배송',
  '건과일',
  '차액',
  '지급',
  '알',
  '있',
  '적립',
  '금액',
  '이해',
  '안',
  '돼서',
  '되',
  '같',
  '여러',
  '번',
  '질문',
  '기다려야',
  '기다리',
  '하',
  '상황',
  '매우',
  '불편'],
 ['한', '개', '상했', '상하']]

In [2]:
# class Documents:
#     def __init__(self, path):
#         self.path = path
#     def __iter__(self):
#         with open(self.path, encoding='utf-8') as f:
#             for doc in f:
#                 yield doc.strip().split()
# 
# documents = Documents(corpus_path)


In [21]:
import gensim

dictionary = gensim.corpora.Dictionary(documents)
print('dictionary size : %d' % len(dictionary)) # dictionary size


dictionary size : 25736


In [22]:
from collections import Counter

min_count = 15
word_counter = Counter((word for words in documents for word in words))
removal_word_idxs = {
    dictionary.token2id[word] for word, count in word_counter.items()
    if count < min_count
}

dictionary.filter_tokens(removal_word_idxs)
dictionary.compactify()
print('dictionary size : %d' % len(dictionary)) # dictionary size : 10354


dictionary size : 4524


In [40]:
# class Corpus:
#     def __init__(self, path, dictionary):
#         self.path = path
#         self.dictionary = dictionary
#         self.length = 0
#     def __iter__(self):
#         with open(self.path, encoding='utf-8') as f:
#             for doc in f:
#                 yield self.dictionary.doc2bow(doc.split())
#     def __len__(self):
#         if self.length == 0:
#             with open(self.path, encoding='utf-8') as f:
#                 for i, doc in enumerate(f):
#                     continue
#             self.length = i + 1
#         return self.length
# 
# corpus = Corpus(corpus_path, dictionary)
# for i, doc in enumerate(corpus):
#     if i >= 5: break
#     print(doc)

In [24]:
corpus = [dictionary.doc2bow(doc) for doc in documents]
for d, c in zip(documents[:3], corpus[:3]):
    print(d, c)


['특상', '주문', '중상', '오배송', '건과일', '차액', '지급', '알', '있', '적립', '금액', '이해', '안', '돼서', '되', '같', '여러', '번', '질문', '기다려야', '기다리', '하', '상황', '매우', '불편'] [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]
['한', '개', '상했', '상하'] [(25, 1), (26, 1), (27, 1), (28, 1)]
['주문취소', '주문', '취소', '싶'] [(18, 1), (29, 1), (30, 1), (31, 1)]


In [ ]:
# import gensim
# from gensim.corpora.dictionary import Dictionary
# 
# corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)
# dictionary = Dictionary.from_corpus(
#     corpus,
#     id2word = dict(enumerate(index2word))
# )

In [41]:
from scipy.sparse import csr_matrix
import numpy as np
x = np.array(corpus)
index2word = dictionary.id2token
word2index = dictionary.token2id


In [43]:
# import gensim
# from gensim.corpora.dictionary import Dictionary
# 
# corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)
# dictionary = Dictionary.from_corpus(
#     corpus,
#     id2word = dict(enumerate(index2word))
# )

In [57]:
from gensim.models import LdaModel
import pickle
lda_model_path = 'LDA_model_190415.pkl'
lda_model = LdaModel(corpus, id2word=index2word, num_topics=20)
with open(lda_model_path, 'wb') as f:
    pickle.dump(lda_model, f)

In [58]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [59]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0


(20,)
1.0
(20, 4524)
1.0


In [60]:
import pyLDAvis.gensim as gensimvis
prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)


/home/admin-/anaconda3/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [62]:
import pyLDAvis
pyLDAvis.display(prepared_data)
